# Phase 2.1 — Base Model Evaluation (Qwen 2.5-7B, No Fine-Tuning)

**What this notebook does:**
1. Loads Qwen 2.5-7B-Instruct in 4-bit quantization (saves ~75% VRAM)
2. Feeds 1,000 stratified training examples (500 credit-relevant, 500 not)
3. Parses every output with our strict parser
4. Reports: parse success rate, per-field accuracy, failure mode taxonomy
5. Prints a GO/NO-GO decision for proceeding to LoRA training

**Why this matters before training:**
If the base model can't even produce our output format >20% of the time, fine-tuning
will waste $50-100 fighting format compliance instead of learning credit signals.
This test establishes a performance floor — any fine-tuned model must beat these numbers.

**Expected outcome:** Base model will probably get <80% parse rate (it's never seen our
format). But even 40-60% means the format is learnable. <20% means we should simplify.

**Time estimate:** ~30-45 min on T4, ~15-20 min on A100 (sequential inference).

**Data location:** `drive/MyDrive/india-credit-signals/data/processed/`

---
## Cell 1: Setup — GPU, Packages, Data

In [ ]:
# 🎓 Step 1a: Verify GPU allocation
# Colab assigns you a GPU when you select Runtime → Change runtime type → GPU.
# T4 (15GB VRAM) is the free tier default. A100 (40GB) is Colab Pro.
# With 4-bit quantization, Qwen 7B needs ~5GB VRAM — T4 is fine.
!nvidia-smi

import torch
print(f"\nPyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_mem = torch.cuda.get_device_properties(0).total_mem / 1e9
    print(f"GPU: {gpu_name} ({gpu_mem:.1f} GB)")
else:
    raise RuntimeError("No GPU! Go to Runtime → Change runtime type → select GPU")

In [ ]:
# 🎓 Step 1b: Install dependencies
# We use pip here (not uv) because Colab's pre-built environment conflicts with uv venvs.
# bitsandbytes: handles 4-bit quantization (QLoRA-style weight compression)
# accelerate: HuggingFace's multi-GPU/mixed-precision toolkit (needed by transformers)
# tqdm: progress bars for the inference loop
!pip install -q transformers>=4.48.0 accelerate>=1.0.0 bitsandbytes>=0.43.0 tqdm

import transformers, accelerate, bitsandbytes
print(f"transformers: {transformers.__version__}")
print(f"accelerate: {accelerate.__version__}")
print(f"bitsandbytes: {bitsandbytes.__version__}")
print("✅ Dependencies installed")

In [ ]:
# 🎓 Step 1c: Mount Google Drive and locate data files
# Google Drive is the bridge between your local Mac (where Claude Code runs)
# and Colab (where GPU inference runs). Push data to Drive, pull results back.
from google.colab import drive
drive.mount('/content/drive')

import os
DATA_DIR = "/content/drive/MyDrive/india-credit-signals/data/processed"
assert os.path.isdir(DATA_DIR), f"Data directory not found: {DATA_DIR}\nUpload your data to Google Drive first."

# Verify key files exist
for fname in ["train.jsonl", "val.jsonl", "test.jsonl", "entity_holdout.jsonl"]:
    fpath = os.path.join(DATA_DIR, fname)
    if os.path.exists(fpath):
        # Count lines
        with open(fpath) as f:
            n = sum(1 for _ in f)
        print(f"  ✅ {fname}: {n:,d} examples")
    else:
        print(f"  ❌ {fname}: NOT FOUND")

print(f"\n✅ Data directory: {DATA_DIR}")

In [ ]:
# 🎓 Step 1d: Load Qwen 2.5-7B-Instruct in 4-bit quantization
#
# ┌─────────────────────────────────────────────────────────┐
# │ 🎓 CONCEPT: 4-bit Quantization (QLoRA-style)            │
# │                                                          │
# │ A 7B model normally needs ~14GB in fp16 (2 bytes/param). │
# │ 4-bit quantization compresses each weight to 4 bits,     │
# │ reducing memory to ~3.5-5GB.                             │
# │                                                          │
# │ NF4 (Normal Float 4) is a special 4-bit format designed  │
# │ for neural network weights. It maps the 16 possible 4-   │
# │ bit values to the most common weight values, minimizing   │
# │ quantization error.                                      │
# │                                                          │
# │ "double_quant" quantizes the quantization constants      │
# │ themselves — saves another ~0.4GB for free.              │
# │                                                          │
# │ Quality loss: ~1-2% accuracy drop vs fp16. Acceptable    │
# │ for testing; we'll use fp16 for final training.          │
# │                                                          │
# │ Paper: https://arxiv.org/abs/2305.14314                  │
# └─────────────────────────────────────────────────────────┘

import time
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"

# 🎓 BitsAndBytesConfig controls the quantization strategy.
# load_in_4bit=True: compress all weights to 4 bits
# bnb_4bit_quant_type="nf4": use NormalFloat4 (better than uniform 4-bit)
# bnb_4bit_compute_dtype=float16: do actual math in fp16 (not 4-bit)
# bnb_4bit_use_double_quant=True: quantize the scale factors too (saves memory)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

print(f"Loading {MODEL_NAME} in 4-bit...")
t0 = time.time()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",  # 🎓 Places model layers across available GPU(s) automatically
)

load_time = time.time() - t0
param_count = sum(p.numel() for p in model.parameters())
gpu_mem_used = torch.cuda.memory_allocated() / 1e9

print(f"✅ Loaded in {load_time:.1f}s")
print(f"   Parameters: {param_count / 1e9:.2f}B")
print(f"   GPU memory used: {gpu_mem_used:.1f} GB")
print(f"   Quantization: 4-bit NF4 with double quantization")

---
## Cell 2: Load & Sample Training Data

In [ ]:
import json
import random

def load_jsonl(path):
    """Load JSONL file into list of dicts."""
    records = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:
                records.append(json.loads(line))
    return records

# Load training data
train_data = load_jsonl(os.path.join(DATA_DIR, "train.jsonl"))
print(f"Loaded {len(train_data):,d} training examples")

# 🎓 Stratified sampling: 50% credit-relevant, 50% not.
# This gives equal coverage of full-form output (6 fields + END) and
# short-form output (2 fields + END). Both formats need to parse correctly.
# If we sampled randomly, we'd get ~54% credit-relevant (natural distribution),
# which is close to 50/50 anyway — but explicit stratification is cleaner.

SAMPLE_SIZE = 1000
SEED = 42
rng = random.Random(SEED)

cr_yes = [ex for ex in train_data if ex.get("output", "").startswith("CREDIT_RELEVANT: Yes")]
cr_no = [ex for ex in train_data if ex.get("output", "").startswith("CREDIT_RELEVANT: No")]

half = SAMPLE_SIZE // 2
sample_yes = rng.sample(cr_yes, min(half, len(cr_yes)))
sample_no = rng.sample(cr_no, min(SAMPLE_SIZE - len(sample_yes), len(cr_no)))
sample = sample_yes + sample_no
rng.shuffle(sample)

print(f"\nSampled {len(sample)} examples:")
print(f"  Credit-relevant (full-form):  {len(sample_yes)}")
print(f"  Not relevant (short-form):    {len(sample_no)}")

# Show one example so you know what the model will see
print("\n" + "=" * 70)
print("EXAMPLE INPUT (first 500 chars):")
print("=" * 70)
print(sample[0]["input"][:500])
print("\nEXPECTED OUTPUT:")
print(sample[0]["output"])

---
## Cell 3: Inference — Run Base Model on 1,000 Examples

🎓 **What's happening here:** We send each example's `instruction + input` to Qwen 2.5-7B
and collect whatever it generates. The model has NEVER seen our output format before.
It will try its best based on general instruction-following ability, but it hasn't been
trained on our specific structured text format. That's the whole point — measuring the gap
between "general intelligence" and "trained for our task."

In [ ]:
from tqdm import tqdm

# 🎓 WHY sequential and not batched: 4-bit quantized models on Colab are
# memory-constrained. Batching requires padding all inputs to max length,
# which wastes VRAM on our variable-length articles. Sequential with KV-cache
# is simpler and more reliable. At ~1-2s per example, 1,000 takes ~20-30 min.

# 🎓 max_new_tokens=200: Our full-form output is ~7 lines (~100 tokens).
# Short-form is ~3 lines (~30 tokens). 200 gives generous headroom for
# verbose base models without wasting too much time on runaway generation.
MAX_NEW_TOKENS = 200

raw_outputs = []
inference_times = []

print(f"Running inference on {len(sample)} examples...")
print(f"Estimated time: ~{len(sample) * 2 / 60:.0f} min on T4, ~{len(sample) * 1 / 60:.0f} min on A100")
print()

for i, example in enumerate(tqdm(sample, desc="Inference")):
    # Build the prompt using Qwen's chat template
    # 🎓 Qwen 2.5 Instruct uses a chat format with <|im_start|> tokens.
    # apply_chat_template handles this automatically. We put the instruction
    # in the system message and the article in the user message.
    messages = [
        {"role": "system", "content": example["instruction"]},
        {"role": "user", "content": example["input"]},
    ]

    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,  # 🎓 Adds the assistant turn start token
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    t0 = time.time()
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=MAX_NEW_TOKENS,
            do_sample=False,       # 🎓 Greedy decoding: deterministic, reproducible
            pad_token_id=tokenizer.eos_token_id,
            # 🎓 Stop at END token or common stop sequences
            # The model may not know to stop at "END", so we also set eos
        )
    gen_time = time.time() - t0
    inference_times.append(gen_time)

    # Extract only the newly generated tokens (not the prompt)
    new_tokens = outputs[0][inputs["input_ids"].shape[1]:]
    response = tokenizer.decode(new_tokens, skip_special_tokens=True).strip()
    raw_outputs.append(response)

    # Show progress every 100 examples
    if (i + 1) % 100 == 0:
        avg_time = sum(inference_times[-100:]) / len(inference_times[-100:])
        remaining = (len(sample) - i - 1) * avg_time / 60
        tqdm.write(f"  [{i+1}/{len(sample)}] avg {avg_time:.1f}s/example, ~{remaining:.0f} min remaining")

total_time = sum(inference_times)
print(f"\n✅ Inference complete: {total_time / 60:.1f} min total, {total_time / len(sample):.1f}s avg")

In [ ]:
# Save raw outputs to Drive (crash-safe — don't lose 30 min of inference)
output_path = os.path.join(DATA_DIR, "base_model_outputs.jsonl")
with open(output_path, "w", encoding="utf-8") as f:
    for example, raw_out in zip(sample, raw_outputs):
        record = {
            "input": example["input"][:200],  # Truncated for storage
            "expected_output": example["output"],
            "model_output": raw_out,
        }
        f.write(json.dumps(record, ensure_ascii=False) + "\n")

print(f"✅ Saved {len(raw_outputs)} outputs to {output_path}")

# Show a few raw outputs so you can see what the model actually produces
print("\n" + "=" * 70)
print("SAMPLE RAW OUTPUTS (first 5):")
print("=" * 70)
for i in range(min(5, len(raw_outputs))):
    print(f"\n--- Example {i+1} ---")
    print(f"Expected: {sample[i]['output'][:100]}...")
    print(f"Got:      {raw_outputs[i][:200]}")

---
## Cell 4: Parse Evaluation — Strict Parser + Failure Taxonomy

🎓 **Source of truth for this parser:** `src/data/parse_training_output.py`  
The code below is copied from that file to make this notebook self-contained on Colab.  
If you change the parser, update it in the source file first, then copy here.

In [ ]:
# ============================================================
# STRICT OUTPUT PARSER
# 🎓 SOURCE OF TRUTH: src/data/parse_training_output.py
# This is a copy for Colab self-containment. If you modify the
# parser, update the source file first, then copy changes here.
# The canonical evaluation module is src/training/evaluate.py.
# ============================================================

import re
from dataclasses import dataclass, field as dataclass_field
from collections import Counter, defaultdict

# Vocabulary — must match configs/training_config.yaml
VOCAB = {
    "credit_relevant": {"Yes", "No"},
    "direction": {"Deterioration", "Improvement", "Neutral"},
    "signal_type": {
        "liquidity", "asset_quality", "regulatory", "contagion",
        "governance", "funding", "operational", "other",
    },
    "sector_wide": {"Yes", "No"},
    "confidence": {"Low", "Medium", "High"},
}


@dataclass
class ParsedOutput:
    """Result of parsing a model's structured text output."""
    credit_relevant: bool = False
    direction: str = ""
    signal_type: str = ""
    sector_wide: bool = False
    confidence: str = ""
    reasoning: str = ""
    has_end_token: bool = False
    parse_ok: bool = False
    error_field: str = ""
    error_detail: str = ""


def parse_model_output(text, vocab=None):
    """Parse structured text output into a ParsedOutput.

    Handles both full-form (credit-relevant) and short-form (not credit-relevant).
    """
    if vocab is None:
        vocab = VOCAB

    result = ParsedOutput()
    text = text.strip()

    # Check for END token
    if text.endswith("END"):
        result.has_end_token = True
        text = text[:-3].strip()
    else:
        result.error_field = "END"
        result.error_detail = "Missing END token"

    # Parse line by line into field → value dict
    fields = {}
    current_key = None
    current_val = ""

    for line in text.split("\n"):
        line = line.strip()
        if not line:
            continue

        matched = False
        for field_name in ("CREDIT_RELEVANT", "DIRECTION", "SIGNAL_TYPE",
                           "SECTOR_WIDE", "CONFIDENCE", "REASONING"):
            prefix = field_name + ":"
            if line.upper().startswith(prefix.upper()):
                if current_key is not None:
                    fields[current_key] = current_val.strip()
                current_key = field_name
                current_val = line[len(prefix):].strip()
                matched = True
                break

        if not matched and current_key is not None:
            current_val += " " + line

    if current_key is not None:
        fields[current_key] = current_val.strip()

    # --- Validate CREDIT_RELEVANT ---
    cr_val = fields.get("CREDIT_RELEVANT", "").strip()
    if not cr_val:
        result.error_field = "CREDIT_RELEVANT"
        result.error_detail = "Missing CREDIT_RELEVANT field"
        return result

    if cr_val not in vocab.get("credit_relevant", {"Yes", "No"}):
        result.error_field = "CREDIT_RELEVANT"
        result.error_detail = f"Invalid value '{cr_val}', expected Yes/No"
        return result

    result.credit_relevant = (cr_val == "Yes")

    # --- Short-form path ---
    if not result.credit_relevant:
        reasoning = fields.get("REASONING", "").strip()
        if not reasoning:
            result.error_field = "REASONING"
            result.error_detail = "Missing REASONING for non-credit-relevant article"
            return result
        result.reasoning = reasoning
        if result.has_end_token:
            result.parse_ok = True
        return result

    # --- Full-form path ---
    dir_val = fields.get("DIRECTION", "").strip()
    if not dir_val:
        result.error_field = "DIRECTION"
        result.error_detail = "Missing DIRECTION field for credit-relevant article"
        return result
    if dir_val not in vocab.get("direction", set()):
        result.error_field = "DIRECTION"
        result.error_detail = f"Invalid value '{dir_val}', expected one of: {sorted(vocab.get('direction', set()))}"
        return result
    result.direction = dir_val

    st_val = fields.get("SIGNAL_TYPE", "").strip()
    if not st_val:
        result.error_field = "SIGNAL_TYPE"
        result.error_detail = "Missing SIGNAL_TYPE field for credit-relevant article"
        return result
    if st_val not in vocab.get("signal_type", set()):
        result.error_field = "SIGNAL_TYPE"
        result.error_detail = f"Invalid value '{st_val}', expected one of: {sorted(vocab.get('signal_type', set()))}"
        return result
    result.signal_type = st_val

    sw_val = fields.get("SECTOR_WIDE", "").strip()
    if not sw_val:
        result.error_field = "SECTOR_WIDE"
        result.error_detail = "Missing SECTOR_WIDE field for credit-relevant article"
        return result
    if sw_val not in vocab.get("sector_wide", {"Yes", "No"}):
        result.error_field = "SECTOR_WIDE"
        result.error_detail = f"Invalid value '{sw_val}', expected Yes/No"
        return result
    result.sector_wide = (sw_val == "Yes")

    conf_val = fields.get("CONFIDENCE", "").strip()
    if not conf_val:
        result.error_field = "CONFIDENCE"
        result.error_detail = "Missing CONFIDENCE field for credit-relevant article"
        return result
    if conf_val not in vocab.get("confidence", set()):
        result.error_field = "CONFIDENCE"
        result.error_detail = f"Invalid value '{conf_val}', expected one of: {sorted(vocab.get('confidence', set()))}"
        return result
    result.confidence = conf_val

    reasoning = fields.get("REASONING", "").strip()
    if not reasoning:
        result.error_field = "REASONING"
        result.error_detail = "Missing REASONING field for credit-relevant article"
        return result
    result.reasoning = reasoning

    if result.has_end_token:
        result.parse_ok = True

    return result


def classify_failure(raw_output, parsed):
    """Classify a parse failure into a taxonomy bucket."""
    text = raw_output.strip()

    # Check for AI refusals
    refusal_patterns = [
        r"(?i)as an (ai|language model|assistant)",
        r"(?i)i cannot", r"(?i)i'm not able to",
        r"(?i)i don't have", r"(?i)i apologize",
        r"(?i)sorry,?\s+(?:but\s+)?i",
    ]
    for pattern in refusal_patterns:
        if re.search(pattern, text):
            return "refusal"

    # Check if ANY known field names appear
    known_fields = {"CREDIT_RELEVANT", "DIRECTION", "SIGNAL_TYPE",
                    "SECTOR_WIDE", "CONFIDENCE", "REASONING"}
    found_fields = set()
    for field_name in known_fields:
        if field_name + ":" in text.upper():
            found_fields.add(field_name)

    if not found_fields:
        return "totally_unstructured"

    # Missing END only
    if parsed.error_field == "END" and parsed.error_detail == "Missing END token":
        test_parsed = parse_model_output(text + "\nEND")
        if test_parsed.parse_ok:
            return "missing_end"

    # Wrong vocab
    if "Invalid value" in parsed.error_detail:
        return "wrong_vocab"

    # Missing field
    if "Missing" in parsed.error_detail and parsed.error_field in known_fields:
        if len(found_fields) >= 2:
            return "partial_format"
        return "missing_field"

    # Extra content
    lines = text.split("\n")
    non_field_lines = []
    for line in lines:
        ls = line.strip()
        if not ls or ls == "END":
            continue
        is_field = any(ls.upper().startswith(f + ":") for f in known_fields)
        if not is_field:
            non_field_lines.append(ls)
    if non_field_lines and len(found_fields) >= 3:
        return "extra_content"

    return "partial_format"


print("✅ Parser and failure classifier defined")

In [ ]:
# Run the strict parser on every output
parsed_results = [parse_model_output(raw) for raw in raw_outputs]

total = len(parsed_results)
parsed_ok = sum(1 for p in parsed_results if p.parse_ok)
failed = total - parsed_ok
parse_rate = parsed_ok / total if total > 0 else 0.0

print("=" * 70)
print("PARSE RESULTS")
print("=" * 70)
print(f"  Total examples:      {total:>6,d}")
print(f"  Successfully parsed: {parsed_ok:>6,d}  ({parse_rate:.1%})")
print(f"  Parse failures:      {failed:>6,d}  ({1 - parse_rate:.1%})")
print()

# Failure mode taxonomy
failure_counts = Counter()
failure_examples = defaultdict(list)

for pred, raw in zip(parsed_results, raw_outputs):
    if not pred.parse_ok:
        bucket = classify_failure(raw, pred)
        failure_counts[bucket] += 1
        if len(failure_examples[bucket]) < 3:
            truncated = raw[:300] + "..." if len(raw) > 300 else raw
            failure_examples[bucket].append(truncated)

if failure_counts:
    print("FAILURE MODE TAXONOMY:")
    print("-" * 70)
    for bucket, count in failure_counts.most_common():
        pct_of_failures = count / failed * 100 if failed > 0 else 0
        pct_of_total = count / total * 100
        print(f"\n  {bucket:25s}  {count:>5d}  ({pct_of_failures:5.1f}% of failures, {pct_of_total:4.1f}% of total)")
        for j, example in enumerate(failure_examples[bucket], 1):
            print(f"    Example {j}:")
            # Indent each line of the example for readability
            for line in example.split("\n")[:5]:  # Max 5 lines per example
                print(f"      {line}")
else:
    print("No parse failures! (Unexpected for a base model — double-check the parser.)")

In [ ]:
# Per-field accuracy (only for successfully parsed outputs)
# 🎓 Even if the model gets the FORMAT right, it might get the CONTENT wrong.
# A model that always says "CREDIT_RELEVANT: Yes" with correct format would
# have 100% parse rate but ~50% credit_relevant accuracy. Both metrics matter.

if parsed_ok > 0:
    # Compare parsed predictions to ground truth
    field_metrics = {
        "credit_relevant": {"total": 0, "correct": 0},
        "direction": {"total": 0, "correct": 0},
        "signal_type": {"total": 0, "correct": 0},
        "sector_wide": {"total": 0, "correct": 0},
        "confidence": {"total": 0, "correct": 0},
    }

    for pred, gt_dict in zip(parsed_results, sample):
        if not pred.parse_ok:
            continue

        # Parse ground truth
        gt = parse_model_output(gt_dict["output"])
        if not gt.parse_ok:
            continue

        # credit_relevant
        field_metrics["credit_relevant"]["total"] += 1
        if pred.credit_relevant == gt.credit_relevant:
            field_metrics["credit_relevant"]["correct"] += 1

        # Only compare other fields when both say credit-relevant
        if pred.credit_relevant and gt.credit_relevant:
            for field_name, pred_val, gt_val in [
                ("direction", pred.direction, gt.direction),
                ("signal_type", pred.signal_type, gt.signal_type),
                ("sector_wide", pred.sector_wide, gt.sector_wide),
                ("confidence", pred.confidence, gt.confidence),
            ]:
                field_metrics[field_name]["total"] += 1
                if pred_val == gt_val:
                    field_metrics[field_name]["correct"] += 1

    print("\n" + "=" * 70)
    print("PER-FIELD ACCURACY (on successfully parsed outputs only)")
    print("=" * 70)
    print(f"  {'Field':20s}  {'Total':>6s}  {'Correct':>7s}  {'Accuracy':>8s}")
    print("-" * 50)
    for field_name, counts in field_metrics.items():
        if counts["total"] > 0:
            acc = counts["correct"] / counts["total"]
            print(f"  {field_name:20s}  {counts['total']:>6d}  {counts['correct']:>7d}  {acc:>8.1%}")
        else:
            print(f"  {field_name:20s}  {'N/A':>6s}  {'N/A':>7s}  {'N/A':>8s}")

    # 🎓 KEY INSIGHT: For a base model, field accuracy tells you how much
    # the model already "understands" about credit analysis vs how much
    # it needs to learn. If credit_relevant accuracy is ~50%, the model is
    # basically random. If direction accuracy is high but signal_type is low,
    # the model gets the big picture but lacks domain vocabulary.
else:
    print("\nNo successfully parsed outputs — cannot compute field accuracy.")
    print("This is expected if parse rate is very low.")

---
## Cell 5: Holdout-Aware Evaluation Scaffold

🎓 **This cell is a scaffold for post-training evaluation.** We define the per-entity
holdout evaluation function here but DON'T run it on base model outputs (because the
base model will be evaluated on train.jsonl samples, not entity_holdout.jsonl).

After LoRA training (Phase 2.3), you'll run the fine-tuned model on `entity_holdout.jsonl`
and use this function to get per-entity metrics.

**Why per-entity?**
- DHFL: 1,243 articles, 91% deterioration — a "dumb" model that always says deterioration scores well
- Reliance Capital: 688 articles, 80% deterioration — similar but different crisis arc
- Cholamandalam: 1,372 articles, 12% deterioration — the real test (false positive control)

Aggregate holdout metrics hide Cholamandalam's failure behind DHFL's easy wins.

In [ ]:
# 🎓 SOURCE OF TRUTH: src/training/evaluate.py
# This function is copied from the canonical evaluation module.
# Post-training, import directly: from src.training.evaluate import evaluate_holdout_per_entity

HOLDOUT_ENTITIES = {
    "DHFL": {"articles": 1243, "det_pct": 91.3, "purpose": "Crisis detection on unseen entity"},
    "Reliance Capital": {"articles": 688, "det_pct": 80.2, "purpose": "Different crisis arc — generalization"},
    "Cholamandalam": {"articles": 1372, "det_pct": 11.9, "purpose": "False positive control on stable entity"},
}


def extract_entity_from_input(input_text):
    """Extract entity name from training input text.
    Input format: 'Entity: DHFL\nDate: 2019-06-01\nTitle: ...\nArticle: ...'
    """
    for line in input_text.split("\n"):
        if line.startswith("Entity:"):
            return line[len("Entity:"):].strip()
    return "UNKNOWN"


def evaluate_holdout_per_entity(predictions, raw_outputs, ground_truth, entities):
    """Evaluate model performance per holdout entity.

    Args:
        predictions: list of ParsedOutput
        raw_outputs: list of raw model output strings
        ground_truth: list of dicts with 'output' field
        entities: list of entity names (from extract_entity_from_input)

    Returns:
        Dict mapping entity name → metrics dict.
    """
    entity_groups = defaultdict(list)
    for i, entity in enumerate(entities):
        entity_groups[entity].append(i)

    results = {}
    for entity, indices in entity_groups.items():
        total = len(indices)
        parsed_indices = [i for i in indices if predictions[i].parse_ok]
        parsed = len(parsed_indices)
        parse_rate = parsed / total if total > 0 else 0.0

        # Failure modes
        failure_modes = Counter()
        for i in indices:
            if not predictions[i].parse_ok:
                failure_modes[classify_failure(raw_outputs[i], predictions[i])] += 1

        # Per-field accuracy
        cr_correct = dir_correct = st_correct = 0
        dir_total = st_total = 0
        det_tp = det_fp = det_fn = 0
        imp_tp = imp_fp = imp_fn = 0

        for i in parsed_indices:
            gt = parse_model_output(ground_truth[i].get("output", ""))
            if not gt.parse_ok:
                continue

            if predictions[i].credit_relevant == gt.credit_relevant:
                cr_correct += 1

            if predictions[i].credit_relevant and gt.credit_relevant:
                dir_total += 1
                if predictions[i].direction == gt.direction:
                    dir_correct += 1
                st_total += 1
                if predictions[i].signal_type == gt.signal_type:
                    st_correct += 1

            # Direction-specific
            pred_det = predictions[i].credit_relevant and predictions[i].direction == "Deterioration"
            gt_det = gt.credit_relevant and gt.direction == "Deterioration"
            pred_imp = predictions[i].credit_relevant and predictions[i].direction == "Improvement"
            gt_imp = gt.credit_relevant and gt.direction == "Improvement"

            if pred_det and gt_det: det_tp += 1
            elif pred_det and not gt_det: det_fp += 1
            elif not pred_det and gt_det: det_fn += 1
            if pred_imp and gt_imp: imp_tp += 1
            elif pred_imp and not gt_imp: imp_fp += 1
            elif not pred_imp and gt_imp: imp_fn += 1

        results[entity] = {
            "total": total,
            "parsed": parsed,
            "parse_rate": parse_rate,
            "cr_accuracy": cr_correct / parsed if parsed > 0 else 0.0,
            "dir_accuracy": dir_correct / dir_total if dir_total > 0 else 0.0,
            "st_accuracy": st_correct / st_total if st_total > 0 else 0.0,
            "det_recall": det_tp / (det_tp + det_fn) if (det_tp + det_fn) > 0 else 0.0,
            "det_precision": det_tp / (det_tp + det_fp) if (det_tp + det_fp) > 0 else 0.0,
            "imp_recall": imp_tp / (imp_tp + imp_fn) if (imp_tp + imp_fn) > 0 else 0.0,
            "imp_precision": imp_tp / (imp_tp + imp_fp) if (imp_tp + imp_fp) > 0 else 0.0,
            "failure_modes": dict(failure_modes),
        }

    return results


def print_holdout_report(entity_metrics):
    """Print per-entity holdout evaluation table."""
    print("=" * 70)
    print("HOLDOUT ENTITY EVALUATION")
    print("=" * 70)
    for entity, em in entity_metrics.items():
        info = HOLDOUT_ENTITIES.get(entity, {})
        purpose = info.get("purpose", "")
        print(f"\n  {entity} ({em['total']} articles) — {purpose}")
        print(f"  {'─' * 55}")
        print(f"    Parse rate:         {em['parse_rate']:>6.1%}  ({em['parsed']}/{em['total']})")
        print(f"    CR accuracy:        {em['cr_accuracy']:>6.1%}")
        print(f"    Direction accuracy:  {em['dir_accuracy']:>6.1%}")
        print(f"    Signal type acc:    {em['st_accuracy']:>6.1%}")
        print(f"    Det. precision:     {em['det_precision']:>6.1%}")
        print(f"    Det. recall:        {em['det_recall']:>6.1%}")
        print(f"    Imp. precision:     {em['imp_precision']:>6.1%}")
        print(f"    Imp. recall:        {em['imp_recall']:>6.1%}")
        if em["failure_modes"]:
            print(f"    Failure modes: {em['failure_modes']}")


print("✅ Holdout evaluation functions defined")
print()
print("Holdout entities:")
for entity, info in HOLDOUT_ENTITIES.items():
    print(f"  {entity:20s}  {info['articles']:>5,d} articles  ({info['det_pct']:.1f}% det.)  {info['purpose']}")
print()
print("To run holdout evaluation after training:")
print("  1. Load entity_holdout.jsonl")
print("  2. Run fine-tuned model on all 3,303 examples")
print("  3. entities = [extract_entity_from_input(ex['input']) for ex in holdout_data]")
print("  4. results = evaluate_holdout_per_entity(preds, raws, holdout_data, entities)")
print("  5. print_holdout_report(results)")

---
## Cell 6: Decision — GO / INVESTIGATE / NO-GO

In [ ]:
print("=" * 70)
print("PHASE 2.1 — BASE MODEL EVALUATION SUMMARY")
print("=" * 70)
print()
print(f"  Model:          {MODEL_NAME}")
print(f"  Quantization:   4-bit NF4")
print(f"  Sample size:    {total:,d} (stratified 50/50 CR)")
print(f"  Inference time: {sum(inference_times)/60:.1f} min total")
print()

# Parse rate summary
print(f"  ┌─────────────────────────────────────────────┐")
print(f"  │  PARSE SUCCESS RATE: {parse_rate:>6.1%}                │")
print(f"  │  ({parsed_ok:,d} / {total:,d} examples)                  │")
print(f"  └─────────────────────────────────────────────┘")
print()

# Decision logic
if parse_rate >= 0.80:
    decision = "GO"
    print("  🟢 DECISION: GO")
    print("  Parse rate >= 80%. The base model already understands our output format.")
    print("  LoRA training will improve both format compliance and content accuracy.")
    print("  → Proceed to Phase 2.2: LoRA training configuration.")
elif parse_rate >= 0.20:
    decision = "INVESTIGATE"
    print("  🟡 DECISION: INVESTIGATE")
    print(f"  Parse rate {parse_rate:.1%} is between 20-80%. Review the failure taxonomy:")
    print()
    if failure_counts:
        top_failure = failure_counts.most_common(1)[0]
        print(f"  Top failure mode: {top_failure[0]} ({top_failure[1]} cases)")
    print()
    print("  If mostly 'missing_end' or 'wrong_vocab':")
    print("    → Fixable. Proceed to training — the model will learn these patterns.")
    print("  If mostly 'totally_unstructured' or 'refusal':")
    print("    → Consider simplifying the format or adding examples to instruction.")
    print("  If mostly 'partial_format':")
    print("    → The model partially gets it. Training should close the gap.")
else:
    decision = "NO-GO"
    print("  🔴 DECISION: NO-GO")
    print(f"  Parse rate {parse_rate:.1%} < 20%. The format is too unfamiliar.")
    print("  Before training ($50-100), consider:")
    print("    1. Simplify output to fewer fields (just CREDIT_RELEVANT + DIRECTION)")
    print("    2. Switch to JSON output (models are more familiar with JSON)")
    print("    3. Add 2-3 output format examples to the instruction prompt")
    print("    4. Try LLaMA 3.1-8B or Mistral as alternative base models")

print()
print("=" * 70)
print()

# Summary for quick reference
print("┌─────────────────────────────────────────────────────────────────┐")
print(f"│  ✅ DONE: Phase 2.1 — Base model evaluation complete            │")
print(f"│  📊 Parse rate: {parse_rate:.1%} | Decision: {decision:12s}              │")
if parsed_ok > 0 and field_metrics["credit_relevant"]["total"] > 0:
    cr_acc = field_metrics["credit_relevant"]["correct"] / field_metrics["credit_relevant"]["total"]
    print(f"│  📊 CR accuracy: {cr_acc:.1%} (on parsed outputs)                  │")
print(f"│  🎓 KEY INSIGHT: This is the performance FLOOR. Any fine-tuned   │")
print(f"│     model must beat these numbers to justify the training cost.  │")
print(f"│  ⏭️  NEXT: Phase 2.2 — LoRA training config on Colab            │")
print("└─────────────────────────────────────────────────────────────────┘")

In [ ]:
# Clean up GPU memory
del model, tokenizer
torch.cuda.empty_cache()
print("✅ GPU memory freed")